## Import des librairies ##

In [ ]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Bibliothèques pour Cross validation
from sklearn.model_selection import StratifiedShuffleSplit

#Graphes
import seaborn as sns
from IPython.display import Markdown, display


## lecture des données ##

In [ ]:
#Lecture du fichier passagers avec label "Survived", pour le training
train = pd.read_csv('Data/passagers.csv', header = 0, dtype={'Age': np.float64})
#Lecture du fichier Test.csv sans label "Survived", à calculer et envoyer à Kaggle
test  = pd.read_csv('Data/test.csv' , header = 0, dtype={'Age': np.float64})
#Création d'un tableau pour boucler sur toutes les données
full_data = [train, test]
#On lit ici les identifiants du fichiers tests, 
# car ils devront être réutilisés à la fin, dans le fichier pour Kaggle
finalfile_index=test.PassengerId 

# Infos sur train et test
display(Markdown('<span style="color:blue;font-weight:bold">train :</span>'))
train.info()
display(Markdown('<span style="color:blue;font-weight:bold">test :</span>'))
test.info()

# Préparation des features #

## Classe et Genre
Lorsqu'il n'y a pas de données manquantes, on analyse seulement l'effet de la variable sur le taux de survie.

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet de la classe</span>'))
print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Genre</span>'))
print (train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean())

## Famille ##
Regardons l'effet de la taille de la famille ( frères/soeurs/conjoints + enfants/parents) sur la survie.

In [ ]:
#On crée une colonne pour la taille de la famille
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
display(Markdown('<span style="color:blue;font-weight:bold">Effet de la taille de la famille</span>'))
print (train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

La taille de la famille n'est pas très explicative. regroupons les passagers seuls et en famille

In [ ]:
#Pour toutes les données (train et test), On crée une colonne indiquant si le passager est seul
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
display(Markdown('<span style="color:blue;font-weight:bold">Effet de la présence de la famille</span>'))
print (train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())

On voit que le fait d'être en famille augmente le taux de survie.

## Port d'embarquement ##
La variable "Embarked" possède des valeurs nulles, nous allons les compléter par la valeur majoritaire (S).
Ensuite, Affichage de l'impact sur la survie

In [ ]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Port d\'embarquement</span>'))
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean())

## Prix du ticket
Les valeurs manquantes sont remplacés par la valeur médiane.  then we categorize it into 4 ranges.
Ensuite, on crée 4 catégories et on analyse leur effet sur la survie.

In [ ]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
#Découpage en 4 catégories
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Prix du Ticket</span>'))
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean())

##Age ##
we have plenty of missing values in this feature. # generate random numbers between (mean - std) and (mean + std).
then we categorize age into 5 range.
De nombreuses valeurs manquent. On les complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).
Ensuite, on crée 5 catégories et on analyse leur effet sur la survie.

In [ ]:
for dataset in full_data:
    age_avg   = dataset['Age'].mean()
    age_std   = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    # Création et attribution pour chaque valeur nulle d'une valeur entière aléatoire
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)    
    dataset.loc[np.isnan(dataset['Age']),'Age'] = age_null_random_list
    #dataset['Age'] = dataset['Age'].astype(int)
#Découpage en 5 catégories
train['CategoricalAge'] = pd.cut(train['Age'], 5)
display(Markdown('<span style="color:blue;font-weight:bold">Effet de l\'age</span>'))
print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean()) # as index évite la création d'un index

## 7. Nom ##
Le nom permet de retrouve le titre du passager

In [ ]:
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # Si le titre existe, on le renvoie
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

print(pd.crosstab(train['Title'], train['Sex']))

On va créer une valeur "Rare" pour les titre peu fréquent, puis afficher l'effet sur la survie

In [ ]:
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                   'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
display(Markdown('<span style="color:blue;font-weight:bold">Effet du titre</span>'))
print (train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean())

# Mise en forme des données #
Après ces premiers traitements, finissons le traitement créant notamment des valeurs numériques

In [1]:
if 'PassengerId' in train.columns.tolist():
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    for dataset in full_data:
        # Mapping Sex
        dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

        # Mapping titles
        dataset['Title'] = dataset['Title'].map(title_mapping)
        dataset['Title'] = dataset['Title'].fillna(0)

        # Mapping Embarked
        dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

        # Mapping Fare
#         dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
#         dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
#         dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
#         dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
#         dataset['Fare'] = dataset['Fare'].astype(int)

#         # Mapping Age
#         dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
#         dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
#         dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
#         dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
#         dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4

    # Selection des Features. On teste pour pouvoir relancer la cellule sans souci
    drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch', 'FamilySize']
    train = train.drop(drop_elements, axis = 1)
    train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
    test  = test.drop(drop_elements, axis = 1)
    
# Infos sur train et test
display(Markdown('<span style="color:blue;font-weight:bold">train mis en forme :</span>'))
train.info()
display(Markdown('<span style="color:blue;font-weight:bold">test mis en forme :</span>'))
test.info()

NameError: name 'train' is not defined